# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging than the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at a minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [1]:
import sqlite3
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [2]:
import pandas as pd
import requests
import json

In [35]:
cur.execute('''SELECT *
               FROM Matches
               WHERE Season = 2011''')
matches_df = pd.DataFrame(cur.fetchall())
matches_df.columns = [x[0] for x in cur.description]
matches_df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [36]:
cur.execute('''SELECT HomeTeam, SUM(FTHG) AS 'Home_Goals'
               FROM Matches
               WHERE Season = 2011
               GROUP BY HomeTeam''')
home_goals = pd.DataFrame(cur.fetchall())
home_goals.columns = [x[0] for x in cur.description]
home_goals.head()

,HomeTeam,Home_Goals
0,Aachen,15
1,Arsenal,39
2,Aston Villa,20
3,Augsburg,20
4,Bayern Munich,49


In [ ]:
# NO
cur.execute('''SELECT HomeTeam, SUM(FTHG) AS 'Home_Goals'
               FROM Matches
               WHERE Season = 2011
               GROUP BY HomeTeam''')
home_df = pd.DataFrame(cur.fetchall())
home_df.columns = [x[0] for x in cur.description]
home_df 

In [37]:
cur.execute('''SELECT AwayTeam, SUM(FTAG) AS 'Away_Goals'
               FROM Matches
               WHERE Season = 2011
               GROUP BY AwayTeam''')
away_goals = pd.DataFrame(cur.fetchall())
away_goals.columns = [x[0] for x in cur.description]
away_goals.head()

,AwayTeam,Away_Goals
0,Aachen,15
1,Arsenal,35
2,Aston Villa,17
3,Augsburg,16
4,Bayern Munich,28


In [38]:
matches_data = pd.concat([home_goals, away_goals], axis=1)
matches_data.head()

,HomeTeam,Home_Goals,AwayTeam,Away_Goals
0,Aachen,15,Aachen,15
1,Arsenal,39,Arsenal,35
2,Aston Villa,20,Aston Villa,17
3,Augsburg,20,Augsburg,16
4,Bayern Munich,49,Bayern Munich,28


In [39]:
matches_data['Total Goals'] = home_goals['Home_Goals'] + away_goals['Away_Goals']
matches_data.head()

,HomeTeam,Home_Goals,AwayTeam,Away_Goals,Total Goals
0,Aachen,15,Aachen,15,30
1,Arsenal,39,Arsenal,35,74
2,Aston Villa,20,Aston Villa,17,37
3,Augsburg,20,Augsburg,16,36
4,Bayern Munich,49,Bayern Munich,28,77


In [40]:
teams = matches_data[['HomeTeam', 'Total Goals']]
teams.head()

,HomeTeam,Total Goals
0,Aachen,30
1,Arsenal,74
2,Aston Villa,37
3,Augsburg,36
4,Bayern Munich,77


In [41]:
cur.execute('''SELECT HomeTeam, COUNT(FTR) AS 'HomeWins'
               FROM Matches
               WHERE Season = 2011 and FTR = 'H'
               GROUP BY HomeTeam''')
home_wins = pd.DataFrame(cur.fetchall())
home_wins.columns = [x[0] for x in cur.description]
home_wins.head() 

,HomeTeam,HomeWins
0,Aachen,4
1,Arsenal,12
2,Aston Villa,4
3,Augsburg,6
4,Bayern Munich,14


In [42]:
cur.execute('''SELECT AwayTeam, COUNT(FTR) AS 'AwayWins'
               FROM Matches
               WHERE Season = 2011 and FTR = 'A'
               GROUP BY AwayTeam''')
away_wins = pd.DataFrame(cur.fetchall())
away_wins.columns = [x[0] for x in cur.description]
away_wins.head()

,AwayTeam,AwayWins
0,Aachen,2
1,Arsenal,9
2,Aston Villa,3
3,Augsburg,2
4,Bayern Munich,9


In [43]:
wins = pd.concat([home_wins, away_wins], axis=1)
wins.head()

,HomeTeam,HomeWins,AwayTeam,AwayWins
0,Aachen,4,Aachen,2
1,Arsenal,12,Arsenal,9
2,Aston Villa,4,Aston Villa,3
3,Augsburg,6,Augsburg,2
4,Bayern Munich,14,Bayern Munich,9


In [44]:
wins['Total Wins'] = home_wins['HomeWins'] + away_wins['AwayWins']
wins.head()

,HomeTeam,HomeWins,AwayTeam,AwayWins,Total Wins
0,Aachen,4,Aachen,2,6
1,Arsenal,12,Arsenal,9,21
2,Aston Villa,4,Aston Villa,3,7
3,Augsburg,6,Augsburg,2,8
4,Bayern Munich,14,Bayern Munich,9,23


In [45]:
team_wins = wins[['HomeTeam', 'Total Wins']]
team_wins.head()

,HomeTeam,Total Wins
0,Aachen,6
1,Arsenal,21
2,Aston Villa,7
3,Augsburg,8
4,Bayern Munich,23


In [46]:
cur.execute('''SELECT HomeTeam, COUNT(FTR) AS 'HomeLoss'
               FROM Matches
               WHERE Season = 2011 and FTR = 'A'
               GROUP BY HomeTeam''')
home_loss = pd.DataFrame(cur.fetchall())
home_loss.columns = [x[0] for x in cur.description]
home_loss.tail()

,HomeTeam,HomeLoss
49,Werder Bremen,5
50,West Brom,10
51,Wigan,7
52,Wolfsburg,5
53,Wolves,13


In [39]:
cur.execute('''SELECT AwayTeam, COUNT(FTR) AS 'AwayLoss'
               FROM Matches
               WHERE Season = 2011 and FTR = 'H'
               GROUP BY AwayTeam''')
away_loss = pd.DataFrame(cur.fetchall())
away_loss.columns = [x[0] for x in cur.description]
away_loss.tail()

,AwayTeam,AwayLoss
51,Werder Bremen,9
52,West Brom,7
53,Wigan,10
54,Wolfsburg,11
55,Wolves,10


In [41]:
loss = home_loss.merge(away_loss, how='inner', left_on='HomeTeam', right_on='AwayTeam')
loss.tail()
# Note: Have to merge the tables because when inspected the table with concat, I had NaN values.  
# Merging puts them together -- review lesson

,HomeTeam,HomeLoss,AwayTeam,AwayLoss
49,Werder Bremen,5,Werder Bremen,9
50,West Brom,10,West Brom,7
51,Wigan,7,Wigan,10
52,Wolfsburg,5,Wolfsburg,11
53,Wolves,13,Wolves,10


In [43]:
loss['Total Loss'] = home_loss['HomeLoss'] + away_loss['AwayLoss']
loss.head()

,HomeTeam,HomeLoss,AwayTeam,AwayLoss,Total Loss
0,Aachen,7,Aachen,8,15.0
1,Arsenal,3,Arsenal,7,10.0
2,Aston Villa,8,Aston Villa,6,14.0
3,Augsburg,4,Augsburg,8,12.0
4,Bayern Munich,2,Bayern Munich,5,7.0


In [45]:
team_losses = loss[['HomeTeam', 'Total Loss']]
team_losses.head()

,HomeTeam,Total Loss
0,Aachen,15.0
1,Arsenal,10.0
2,Aston Villa,14.0
3,Augsburg,12.0
4,Bayern Munich,7.0


In [ ]:
df = pd.concat()

In [30]:
teams = []
goals = []
wins = []
losses = []
for team in matches_df['HomeTeam'].unique():
    teams.append(team)
    
    goals_H = matches_df['FTHG'][matches_df['HomeTeam']==team].sum()
    goals_A = matches_df['FTAG'][matches_df['AwayTeam']==team].sum()
    goals.append(goals_H + goals_A)
    
    wins_H = matches_df[(matches_df['HomeTeam']==team) & (matches_df['FTR']=='H')]
    wins_A = matches_df[(matches_df['AwayTeam']==team) & (matches_df['FTR']=='A')]
    wins.append(len(wins_H) + len(wins_A))
    
    loss_H = matches_df[(matches_df['HomeTeam']==team) & (matches_df['FTR']=='A')]
    loss_A = matches_df[(matches_df['AwayTeam']==team) & (matches_df['FTR'=='H'])]
    losses.append(len(loss_H) + len(loss_A))
    
df=pd.DataFrame()
df['Team']=teams
df['Goals']=goals
df['Wins']=wins
df['Losses']=losses
df.head(10)                    

KeyError: False

### API

In [19]:
cur.execute('''SELECT HomeTeam, FTHG, FTAG,FTR, Date
               FROM Matches
               WHERE Season = 2011 AND FTR IS NOT 'D';''')
main_df = pd.DataFrame(cur.fetchall())
main_df.columns = [x[0] for x in cur.description]
main_df.head()

,HomeTeam,FTHG,FTAG,FTR,Date
0,Nurnberg,0,1,A,2012-03-31
1,Stuttgart,1,2,A,2011-12-11
2,Wolfsburg,0,1,A,2011-08-13
3,Mainz,3,2,H,2011-11-27
4,M'gladbach,3,1,H,2012-01-20


In [20]:
len(main_df.Date.unique())

152

In [21]:
url = 'https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,'
weather = []
for date in main_df['Date'].unique():
    request_url = url + '{}T12:00:00?exclude=currently,minutely,hourly,alerts,flags'.format(date)
    print(request_url)

https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,2012-03-31T12:00:00?exclude=currently,minutely,hourly,alerts,flags
https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,2011-12-11T12:00:00?exclude=currently,minutely,hourly,alerts,flags
https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,2011-08-13T12:00:00?exclude=currently,minutely,hourly,alerts,flags
https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,2011-11-27T12:00:00?exclude=currently,minutely,hourly,alerts,flags
https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,2012-01-20T12:00:00?exclude=currently,minutely,hourly,alerts,flags
https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,2012-04-21T12:00:00?exclude=currently,minutely,hourly,alerts,flags
https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,2011-09-18T12:00:00?excl

In [23]:
url = 'https://api.darksky.net/forecast/a69c50a13efed54ea880b761d856f6b4/52.5200,13.4050,'
weather = {}
for date in main_df['Date'].unique():
    print('Date Queried: {}'.format(date))
    request_url = url + '{}T12:00:00?exclude=currently,minutely,hourly,alerts,flags'.format(date)
    response = requests.get(request_url).json()
    print(response['daily']['data'][0]['icon'], response['daily']['data'][0]['precipProbability'])
    weather[date] = response['daily']['data'][0]['icon']

Date Queried: 2012-03-31
rain 0.56
Date Queried: 2011-12-11
partly-cloudy-day 0
Date Queried: 2011-08-13
partly-cloudy-day 0
Date Queried: 2011-11-27
partly-cloudy-day 0
Date Queried: 2012-01-20
rain 0.57
Date Queried: 2012-04-21
partly-cloudy-day 0
Date Queried: 2011-09-18
rain 1
Date Queried: 2011-10-23
rain 0.48
Date Queried: 2012-03-03
clear-day 0
Date Queried: 2011-08-27
rain 0.48
Date Queried: 2012-03-17
clear-day 0
Date Queried: 2011-11-06
rain 0.48
Date Queried: 2012-05-05
rain 0.57
Date Queried: 2012-04-11
partly-cloudy-day 0
Date Queried: 2011-12-17
rain 1
Date Queried: 2012-02-18
partly-cloudy-day 0
Date Queried: 2012-02-03
partly-cloudy-day 0
Date Queried: 2012-01-22
rain 0.57
Date Queried: 2012-04-14
clear-day 0
Date Queried: 2012-03-25
partly-cloudy-day 0
Date Queried: 2012-04-07
partly-cloudy-day 0
Date Queried: 2011-11-19
partly-cloudy-day 0
Date Queried: 2011-10-14
partly-cloudy-day 0
Date Queried: 2011-09-24
clear-day 0
Date Queried: 2012-04-28
clear-day 0
Date Querie

In [25]:
weather_list = []
for date in main_df['Date']:
    weather_list.append(weather[date])
main_df['Weather'] = weather_list
main_df.head()

,HomeTeam,FTHG,FTAG,FTR,Date,Weather
0,Nurnberg,0,1,A,2012-03-31,rain
1,Stuttgart,1,2,A,2011-12-11,partly-cloudy-day
2,Wolfsburg,0,1,A,2011-08-13,partly-cloudy-day
3,Mainz,3,2,H,2011-11-27,partly-cloudy-day
4,M'gladbach,3,1,H,2012-01-20,rain


In [28]:
main_df['Did Rain'] = 0
main_df['Did Rain'][main_df['Weather'] == 'rain'] = 1
main_df

/Volumes/Data/Users/jucabrera/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,HomeTeam,FTHG,FTAG,FTR,Date,Weather,Did Rain
0,Nurnberg,0,1,A,2012-03-31,rain,1
1,Stuttgart,1,2,A,2011-12-11,partly-cloudy-day,0
2,Wolfsburg,0,1,A,2011-08-13,partly-cloudy-day,0
3,Mainz,3,2,H,2011-11-27,partly-cloudy-day,0
4,M'gladbach,3,1,H,2012-01-20,rain,1
5,Werder Bremen,1,2,A,2012-04-21,partly-cloudy-day,0
6,Schalke 04,0,2,A,2011-09-18,rain,1
7,Hannover,2,1,H,2011-10-23,rain,1
8,Leverkusen,2,0,H,2012-03-03,clear-day,0
9,Kaiserslautern,0,3,A,2011-08-27,rain,1


In [ ]:
for x in df['TeamName']:
    sum_wins = 0
    sum_losses = 0
    sum_draw = 0
    sum_total = 0
    for y in range(len(df_Matches)):
        if df_Matches.iloc[y]['HomeTeam'] == x:
            if df_Matches.iloc[y]['FTR'] == 'H':
                sum_wins += 1
                sum_total += 1
            elif df_Matches.iloc[y]['FTR'] == 'D':
                sum_draw += 1
                sum_total += 1
            else:
                sum_losses += 1
                sum_total += 1
        elif df_Matches.iloc[y]['AwayTeam'] == x:
            if df_Matches.iloc[y]['FTR'] == 'A':
                sum_wins += 1
                sum_total += 1
            elif df_Matches.iloc[y]['FTR'] == 'D':
                sum_draw += 1
                sum_total += 1
            else:
                sum_losses += 1
                sum_total += 1
        else:
            pass
    print(x)
    print(sum_wins)
    print(sum_losses)
    print(sum_draw)
    print(sum_wins + sum_losses + sum_draw)
    print(sum_total)

In [ ]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    pass

In [ ]:
class MongoHandler():
    pass

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!